In [1]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import snorkel
import dotenv
import glob
import tqdm
from snorkel.parser import TextDocPreprocessor
from snorkel.parser import CorpusParser
from snorkel.models import Candidate, Document, Sentence
from snorkel import SnorkelSession

dotenv.load_dotenv('../env.sh')
%run ../src/supervision.py
collection_dir = osp.join(os.environ['REPO_DATA_DIR'], 'brat', 'collection_02')
session = SnorkelSession()

# Choose whether candidates will be loaded as a train/dev/test (i.e. 0/1/2) split or as a single inference split (3)
#corpus_dir = osp.join(os.environ['DATA_DIR'], 'articles', 'corpus', 'corpus_00')
#candidate_mode='training'

corpus_dir = osp.join(os.environ['DATA_DIR'], 'articles', 'corpus', 'corpus_01')
candidate_mode='inference'

corpus_docs_dir = osp.join(corpus_dir, 'links')
# rm ~/repos/hammer/t-cell-relation-extraction/pm_subtype_protein_relations/snorkel/snorkel.db

In [2]:
# Clear existing candidates, if need be
ct = session.query(Candidate).count()
# if ct > 0:
#     # Clear all existing candidates (don't let extractors do it)
#     # See: https://github.com/HazyResearch/snorkel/blob/master/snorkel/candidates.py#L47
#     # *This seems to always cause a database lock somehow -- perhaps it needs to be done with autocommit but
#     # for now a workaround is to do this at the beginning and the restart the kernel
#     from snorkel.models import Candidate
#     ndelete = session.query(Candidate).delete()
#     session.commit()
#     # Restart kernel
ct

170683

In [3]:
tags = pd.read_csv(osp.join(corpus_dir, 'tags.csv'))
tags = tags[tags['type'].isin(ENT_TYPES)]
print(tags['type'].value_counts())
tags.head()

CYTOKINE                484597
IMMUNE_CELL_TYPE        314554
TRANSCRIPTION_FACTOR    245369
Name: type, dtype: int64


,id,type,ent_id,ent_lbl,ent_prefid,start_chr,end_chr,start_wrd,end_wrd,text
1,PMC5704053,IMMUNE_CELL_TYPE,CT30BC86BDEF7B1410,Treg,CTB574584AD019ABB8,26,38,7,9,Regulatory T
2,PMC5704053,IMMUNE_CELL_TYPE,CT30BC86BDEF7B1410,Treg,CTB574584AD019ABB8,125,137,29,31,regulatory T
3,PMC5704053,IMMUNE_CELL_TYPE,CTB574584AD019ABB8,Treg,CTB574584AD019ABB8,145,149,33,34,Treg
4,PMC5704053,IMMUNE_CELL_TYPE,CTB574584AD019ABB8,Treg,CTB574584AD019ABB8,327,331,63,64,Treg
8,PMC5704053,CYTOKINE,CK379C94E0D2330772,IL-10,CKC5CC1A269C01EC48,468,473,87,88,IL-10


In [4]:
from string import punctuation

def offsets_to_token(left, right, offset_array, lemmas, punc=set(punctuation)):
    token_start, token_end = None, None
    for i, c in enumerate(offset_array):
        if left >= c:
            token_start = i
        if c > right and token_end is None:
            token_end = i
            break
    token_end = len(offset_array) - 1 if token_end is None else token_end
    token_end = token_end - 1 if lemmas[token_end - 1] in punc else token_end
    return range(token_start, token_end)


class EntityTagger(object):

    def __init__(self, tags):   
        self.tags = tags.set_index('id')
        self.reset_stats()

    def reset_stats(self):
        self.stats = {'docs': set(), 'found': set()}
        return self
    
    def get_stats(self):
        return dict(
            n_tags=len(self.tags), 
            n_docs=len(self.stats['docs']),
            n_docs_found=len(self.stats['found']),
            pct_docs_found=100*len(self.stats['found'])/len(self.stats['docs'])
        )
    
    def tag(self, parts):
        """Tag tokens in a single sentence"""
        # Extract doc id (e.g. PMC123932) and character offsets of sentence
        docid, _, _, sent_start, sent_end = parts['stable_id'].split(':')
        self.stats['docs'].add(docid)
        if docid not in self.tags.index:
            return parts
        self.stats['found'].add(docid)
        tags = self.tags.loc[[docid]]
        sent_start, sent_end = int(sent_start), int(sent_end)
        for r in tags.itertuples():
            tag_start, tag_end = r.start_chr, r.end_chr
            # Determine whether or not the tag is in this sentence
            if not (sent_start <= tag_start <= sent_end):
                continue
            offsets = [offset + sent_start for offset in parts['char_offsets']]
            tkn_idx_rng = offsets_to_token(tag_start, tag_end, offsets, parts['lemmas'])
            for tkn_idx in tkn_idx_rng:
                parts['entity_types'][tkn_idx] = r.type.lower()
                parts['entity_cids'][tkn_idx] = r.ent_id + ':' + r.ent_prefid
        return parts

In [5]:
def get_id_from_file(f):
    return osp.splitext(osp.basename(f))[0]

def get_dir_doc_ids(path):
    return set([get_id_from_file(f) for f in glob.glob(osp.join(path, '*.txt'))])

dev_ids = get_dir_doc_ids(collection_dir)
corpus_00_ids = get_dir_doc_ids(osp.join(os.environ['DATA_DIR'], 'articles', 'corpus', 'corpus_00', 'links'))
corpus_01_ids = get_dir_doc_ids(osp.join(os.environ['DATA_DIR'], 'articles', 'corpus', 'corpus_01', 'links'))

train_ids = corpus_00_ids.difference(dev_ids)
inference_ids = corpus_01_ids.difference(corpus_00_ids)
inserted_ids = set([r[0] for r in session.query(Document.name).all()])

# Show frequency of docs that are annotated AND have tags of some kind
#pd.Series({doc_id:doc_id in tags['id'].values for doc_id in dev_ids}).value_counts()
assert len(dev_ids.intersection(train_ids)) == 0
assert len(dev_ids.intersection(inference_ids)) == 0
len(dev_ids), len(train_ids), len(inference_ids), len(inserted_ids)

(89, 487, 9727, 6603)

In [6]:
all_corpus_files = glob.glob(osp.join(corpus_docs_dir, '*.txt'))
corpus_files = [f for f in all_corpus_files if get_id_from_file(f) not in inserted_ids]
len(all_corpus_files), len(corpus_files)

(10043, 3700)

In [7]:
from snorkel.parser import TextDocPreprocessor

class DocListProcessor(TextDocPreprocessor):
    
    def __init__(self, paths, encoding="utf-8"):
        super().__init__(None, encoding=encoding)
        self.paths = paths
                    
    def _get_files(self, path):
        return self.paths
    
#doc_preprocessor = TextDocPreprocessor(corpus_docs_dir, max_docs=10)

In [8]:
batch_size = 100
tagger = EntityTagger(tags)
corpus_parser = CorpusParser(fn=tagger.tag)

# Process in batches since Document/Sentence objects are buffered into memory
# until .commit is called (which is only done at end of UDF .apply)
for batch in tqdm.tqdm(np.array_split(corpus_files, len(corpus_files)//batch_size)):
    doc_preprocessor = DocListProcessor(list(batch))
    corpus_parser.apply(list(doc_preprocessor), clear=False)

  0%|          | 0/100 [00:00<?, ?it/s]

Running UDF...



  0%|          | 0/100 [00:00<?, ?it/s]

Running UDF...



  0%|          | 0/100 [00:00<?, ?it/s]

...


  0%|          | 0/100 [00:00<?, ?it/s]

Running UDF...



  0%|          | 0/100 [00:00<?, ?it/s]

Running UDF...



100%|██████████| 37/37 [3:57:50<00:00, 394.05s/it][A


In [ ]:
# from snorkel.parser import CorpusParser, Spacy

# tagger = EntityTagger(tags)
# corpus_parser = CorpusParser(fn=tagger.tag)
# corpus_parser.apply(list(doc_preprocessor), clear=False)

In [10]:
# This will show how many documents didn't have any tagged
# (or that were otherwise not included in tagging but included
# here in parsing -- which should be rare)
tagger.get_stats()

{'n_tags': 1044520,
 'n_docs': 3700,
 'n_docs_found': 3451,
 'pct_docs_found': 93.27027027027027}

In [11]:
print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Documents: 10303
Sentences: 1907588


In [12]:
docs = session.query(Document).all()

In [13]:
all_ids = set([doc.name for doc in docs])
tagged_ids = set(np.intersect1d(list(all_ids), tags['id']))
len(all_ids), len(tagged_ids)

(10303, 9345)

In [14]:
train_sents, dev_sents, inference_sents = [], [], []
for i, doc in enumerate(docs):
    if doc.name not in tagged_ids:
        continue
    for s in doc.sentences:
        if doc.name in train_ids:
            train_sents.append(s)
        elif doc.name in dev_ids:
            dev_sents.append(s)
        elif doc.name in inference_ids:
            inference_sents.append(s)
        else:
            raise Exception('ID <{0}> not found in any id set'.format(doc.name))
train_sents, dev_sents, inference_sents = set(train_sents), set(dev_sents), set(inference_sents)

In [16]:
from snorkel.candidates import PretaggedCandidateExtractor
classes = get_candidate_classes()
candidate_extractors = [
    PretaggedCandidateExtractor(c.subclass, c.entity_types)
    for c in classes.values()
]

In [46]:
# for extractor in candidate_extractors:
#     extractor.clear(session, split=SPLIT_INFER)
# session.commit()

In [17]:
def apply_extraction(sents, split, batch_size=10000):
    for extractor in candidate_extractors:
        relation_class = extractor.udf_init_kwargs['candidate_class']
        n_batch = len(sents)//batch_size
        print('Beginning candidate extraction for split {}, relation type {}, num batches {}'.format(
            split, relation_class.__name__, n_batch
        ))
        for batch in tqdm.tqdm(np.array_split(list(sents), n_batch)):
            extractor.apply(batch, split=split, clear=False, progress_bar=False)
        print('Number of candidates generated for split {}, relation type {} = {}'.format(
            split, relation_class.__name__,
            session.query(relation_class).filter(relation_class.split == split).count()
        ))

if candidate_mode == 'training':
    for split, sents in enumerate([train_sents, dev_sents]):
        apply_extraction(sents, split)
elif candidate_mode == 'inference':
    apply_extraction(inference_sents, SPLIT_INFER)
else:
    raise ValueError('Candidate mode "{}" not valid'.format(candidate_mode))
    

Beginning candidate extraction for split 3, relation type InducingCytokine, num batches 183


  0%|          | 0/183 [00:00<?, ?it/s]

Running UDF...


  1%|          | 1/183 [00:06<19:07,  6.31s/it]

Running UDF...


...

Running UDF...


 99%|█████████▉| 182/183 [14:33<00:03,  3.14s/it]

Running UDF...


100%|██████████| 183/183 [14:36<00:00,  3.09s/it]


Number of candidates generated for split 3, relation type InducingTranscriptionFactor = 72076


## Load Gold Labels

Only relevant for ```candidate_mode == 'training'```

In [ ]:
if candidate_mode != 'training':
    raise ValueError('Loading manual annotations only relevant for training data')

In [15]:
# Read csv export with annotated relations to load:
relations = pd.read_csv(osp.join(corpus_dir, 'relations.csv'))
relations.head()

,e1_end_chr,e1_start_chr,e1_text,e1_typ,e2_end_chr,e2_start_chr,e2_text,e2_typ,id,rel_typ
0,24,19,Gfi-1,TRANSCRIPTION_FACTOR,85,81,Th17,IMMUNE_CELL_TYPE,PMC2646571,Differentiation
1,44,39,TGF-β,CYTOKINE,85,81,Th17,IMMUNE_CELL_TYPE,PMC2646571,Induction
2,44,39,TGF-β,CYTOKINE,119,97,inducible regulatory T,IMMUNE_CELL_TYPE,PMC2646571,Induction
3,24,19,Gfi-1,TRANSCRIPTION_FACTOR,119,97,inducible regulatory T,IMMUNE_CELL_TYPE,PMC2646571,Differentiation
4,371,366,Gfi-1,TRANSCRIPTION_FACTOR,436,433,Th2,IMMUNE_CELL_TYPE,PMC2646571,Differentiation


In [16]:
relations['rel_typ'].value_counts()

Induction          150
Secretion          131
Differentiation    119
Name: rel_typ, dtype: int64

In [17]:
relations.groupby(['e1_typ', 'e2_typ', 'rel_typ']).size()

e1_typ                e2_typ            rel_typ        
CYTOKINE              IMMUNE_CELL_TYPE  Induction          150
                                        Secretion          131
TRANSCRIPTION_FACTOR  IMMUNE_CELL_TYPE  Differentiation    119
dtype: int64

In [145]:
# # Reset annotation tables (if loading them below fails)
# session.execute('DELETE FROM stable_label;')
# session.execute('DELETE FROM gold_label;')
# session.execute('DELETE FROM gold_label_key;')
# from snorkel.models import GoldLabel, GoldLabelKey, StableLabel
# session.commit()
# session.query(StableLabel).count(), session.query(GoldLabel).count(), session.query(GoldLabelKey).count()

In [18]:
from snorkel.models import StableLabel
from snorkel.db_helpers import reload_annotator_labels

def get_stable_id(r):
    return '{}::span:{}:{}'.format(r['id'], r['start_chr'], r['end_chr']-1)

def reload_labels(
    session, candidate_class, annotator_name, split, 
    filter_label_split=True, create_missing_cands=False):
    """Reloads stable annotator labels into the AnnotatorLabel table"""
    from snorkel.models import GoldLabel, GoldLabelKey, StableLabel, Context
    from future.utils import iteritems
    # Sets up the AnnotatorLabelKey to use
    ak = session.query(GoldLabelKey).filter(GoldLabelKey.name == annotator_name).first()
    if ak is None:
        ak = GoldLabelKey(name=annotator_name)
        session.add(ak)
        session.commit()

    labels = []
    missed = []
    sl_query = session.query(StableLabel).filter(StableLabel.annotator_name == annotator_name)
    sl_query = sl_query.filter(StableLabel.split == split) if filter_label_split else sl_query
    for sl in sl_query.all():
        context_stable_ids = sl.context_stable_ids.split('~~')

        # Check for labeled Contexts
        # TODO: Does not create the Contexts if they do not yet exist!
        contexts = []
        for stable_id in context_stable_ids:
            context = session.query(Context).filter(Context.stable_id == stable_id).first()
            if context:
                contexts.append(context)
        if len(contexts) < len(context_stable_ids):
            missed.append(sl)
            continue

        # Check for Candidate
        # Assemble candidate arguments
        candidate_args  = {'split' : split}
        for i, arg_name in enumerate(candidate_class.__argnames__):
            candidate_args[arg_name] = contexts[i]

        # Assemble query and check
        candidate_query = session.query(candidate_class)
        for k, v in iteritems(candidate_args):
            candidate_query = candidate_query.filter(getattr(candidate_class, k) == v)
        candidate = candidate_query.first()

        # Optionally construct missing candidates
        if candidate is None and create_missing_cands:
            candidate = candidate_class(**candidate_args)

        # If candidate is none, mark as missed and continue
        if candidate is None:
            missed.append(sl)
            continue

        # Check for AnnotatorLabel, otherwise create
        label = session.query(GoldLabel).filter(GoldLabel.key == ak).filter(GoldLabel.candidate == candidate).first()
        if label is None:
            label = GoldLabel(candidate=candidate, key=ak, value=sl.value)
            session.add(label)
            labels.append(label)

    session.commit()
    print("AnnotatorLabels created: %s, missed: %s" % (len(labels), len(missed)))
    return missed, labels
    
def load_external_labels(session, relations, candidate_class, annotator_name='gold'):
    print(annotator_name)
    for i, r in relations.iterrows():    

        # We check if the label already exists, in case this cell was already executed
        e1_id = get_stable_id(r.filter(regex='^e1_|^id$').rename(lambda v: v.replace('e1_', '')))
        e2_id = get_stable_id(r.filter(regex='^e2_|^id$').rename(lambda v: v.replace('e2_', '')))
        
        context_stable_ids = "~~".join([e1_id, e2_id])
        query = session.query(StableLabel)\
            .filter(StableLabel.context_stable_ids == context_stable_ids)\
            .filter(StableLabel.annotator_name == annotator_name)
        if query.count() == 0:
            session.add(StableLabel(
                context_stable_ids=context_stable_ids,
                annotator_name=annotator_name,
                value=1
            ))
            
    session.commit()
    # This function will create GoldLabel records for each StableLabel above after
    # selecting them based on annotator_name.  The annotator name should be different
    # for each candidate class if they might have identical context_stable_ids since
    # otherwise as written above only the first value for the same annotator + context_stable_ids
    # will be saved.
    # Other notes: split will be used to find candidates necessary to create GoldLabels though
    # it is not necessary for StableLabel filtering in this case (thus filter_label_split=False)
    # because the labels were not created with a split above
    #reload_annotator_labels(
    return reload_labels(
        session, candidate_class, annotator_name, split=1, 
        filter_label_split=False, create_missing_cands=False)
    
cand_summary = {}
for extractor in candidate_extractors:
    relation_class = extractor.udf_init_kwargs['candidate_class']
    label = classes[relation_class.__name__].label
    field = classes[relation_class.__name__].field
    df = relations[relations['rel_typ'] == label]
    assert len(df) > 0, 'Found no records for relation type {}'.format(label)
    print('Found {} relations for type {}'.format(len(df), label))
    cand_summary[relation_class.__name__] = load_external_labels(session, df, relation_class, annotator_name=field)

Found 150 relations for type Induction
inducing_cytokine
AnnotatorLabels created: 133, missed: 17
Found 131 relations for type Secretion
secreted_cytokine
AnnotatorLabels created: 81, missed: 50
Found 119 relations for type Differentiation
inducing_transcription_factor
AnnotatorLabels created: 86, missed: 33


In [19]:
from snorkel.models import Context 

# Show which entity types and relations were unable to be matched with spans
# extracted and inserted into snorkel db
def summarize_missing_candidates(cand_summary):
    df = []
    for c in cand_summary:
        class_name = classes[c].name
        missed = cand_summary[c][0]
        for mc in missed:
            doc_id = mc.context_stable_ids.split('::')[0]
            ids = mc.context_stable_ids.split('~~')
            typs = []
            for i, sid in enumerate(ids):
                ctx = session.query(Context).filter(Context.stable_id == sid).all()
                if len(ctx) == 0:
                    typs.append(classes[c].entity_types[i])
            df.append((class_name, doc_id, ','.join(typs)))
    return pd.DataFrame(df, columns=['relation', 'doc_id', 'missing'])
df_miss = summarize_missing_candidates(cand_summary)
df_miss.groupby(['relation', 'missing']).size()

relation                     missing                              
InducingCytokine             cytokine                                  7
                             immune_cell_type                         10
InducingTranscriptionFactor                                            1
                             immune_cell_type                          7
                             transcription_factor                     18
                             transcription_factor,immune_cell_type     7
SecretedCytokine             cytokine                                 11
                             cytokine,immune_cell_type                 3
                             immune_cell_type                         36
dtype: int64

In [20]:
# Find documents with most annotations unable to be matched and either improve tagging or change annotations
df_miss.groupby(['relation', 'doc_id']).size().sort_values(ascending=False).head(15)

relation                     doc_id    
SecretedCytokine             PMC3046151    13
                             PMC2196041    11
InducingTranscriptionFactor  PMC2587175     8
                             PMC2783637     6
                             PMC2646571     5
SecretedCytokine             PMC2193209     5
InducingCytokine             PMC3173465     4
SecretedCytokine             PMC4385920     4
InducingTranscriptionFactor  PMC3173465     4
SecretedCytokine             PMC3650071     4
InducingTranscriptionFactor  PMC3304099     3
                             PMC5591438     2
InducingCytokine             PMC4023883     2
InducingTranscriptionFactor  PMC5206501     2
InducingCytokine             PMC2196041     2
dtype: int64

In [21]:
from snorkel.annotations import load_gold_labels, load_matrix
from snorkel.models import Candidate

L_gold = {}
for c in classes:
    cids_query = get_cids_query(session, classes[c], split=1)
    L_gold[c] = load_gold_labels(session, annotator_name=classes[c].field, split=1, cids_query=cids_query)

In [22]:
for c in L_gold:
    print(c, L_gold[c].shape)

InducingCytokine (673, 1)
SecretedCytokine (673, 1)
InducingTranscriptionFactor (410, 1)


In [23]:
for c in classes.values():
    for split in [0, 1]:
        n = session.query(c.subclass).filter(c.subclass.split == split).count()
        print('Candidate counts: {} (split {}) -> {}'.format(c.name, split, n))

Candidate counts: InducingCytokine (split 0) -> 11735
Candidate counts: InducingCytokine (split 1) -> 673
Candidate counts: SecretedCytokine (split 0) -> 11735
Candidate counts: SecretedCytokine (split 1) -> 673
Candidate counts: InducingTranscriptionFactor (split 0) -> 6696
Candidate counts: InducingTranscriptionFactor (split 1) -> 410


In [24]:
# Make sure that inducing/secreted cytokine labels are mutally exclusive
L_df = pd.DataFrame(np.hstack((
    L_gold[classes.inducing_cytokine.name].toarray(), 
    L_gold[classes.secreted_cytokine.name].toarray()
)))
L_df.groupby([0, 1]).size()

0  1
0  0    459
   1     81
1  0    133
dtype: int64

In [25]:
pd.Series([
    type(L_gold[classes.inducing_cytokine.name].get_candidate(session, i)).__name__ 
    for i in range(L_gold[classes.inducing_cytokine.name].shape[0])
]).value_counts()

InducingCytokine    673
dtype: int64